<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Constucting-Factors" data-toc-modified-id="Constucting-Factors-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Constucting Factors</a></span><ul class="toc-item"><li><span><a href="#Define-factors" data-toc-modified-id="Define-factors-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Define factors</a></span><ul class="toc-item"><li><span><a href="#Quality-factors" data-toc-modified-id="Quality-factors-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Quality factors</a></span></li><li><span><a href="#Growth-factors" data-toc-modified-id="Growth-factors-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Growth factors</a></span></li><li><span><a href="#Momentum-factors" data-toc-modified-id="Momentum-factors-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Momentum factors</a></span></li><li><span><a href="#Volatility-factors" data-toc-modified-id="Volatility-factors-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Volatility factors</a></span></li><li><span><a href="#Size-factors" data-toc-modified-id="Size-factors-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>Size factors</a></span></li><li><span><a href="#Technical-factors" data-toc-modified-id="Technical-factors-1.1.6"><span class="toc-item-num">1.1.6&nbsp;&nbsp;</span>Technical factors</a></span></li><li><span><a href="#Fundamental-factors" data-toc-modified-id="Fundamental-factors-1.1.7"><span class="toc-item-num">1.1.7&nbsp;&nbsp;</span>Fundamental factors</a></span></li></ul></li><li><span><a href="#Get-factor-data" data-toc-modified-id="Get-factor-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Get factor data</a></span><ul class="toc-item"><li><span><a href="#Construct-factors" data-toc-modified-id="Construct-factors-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Construct factors</a></span></li></ul></li></ul></li></ul></div>

## Constucting Factors
Let us recall where we left last time. We have successfully downloaded all the data we need and put them in one folder.

In [23]:
# Get meta data dataframe
meta_df = pd.json_normalize(load_meta())
meta_df['sicCode'].fillna(0, inplace=True)

# Download data
for symbol in notebook.tqdm(stock_list):
    print(f'Start processing {symbol}...')
    download_csv_data(
        symbol=symbol,
        start_date=START_DATE,
        end_date=END_DATE,
        freq='daily',
        path=DATA_PATH
    )

Start processing a...
Start processing aal...
Start processing aap...
Start processing aapl...
Start processing abbv...
Start processing abc...
Start processing abmd...
Start processing abt...
Start processing acn...
Start processing adbe...
Start processing adi...
Start processing adm...
Start processing adp...
Start processing ads...
Start processing adsk...
Start processing aee...
Start processing aep...
Start processing aes...
Start processing aet...
Start processing afl...
Start processing agn...
Start processing aig...
Start processing aiv...
Start processing aiz...
Start processing ajg...
Start processing akam...
Start processing alb...
Start processing algn...
Start processing alk...
Start processing all...
Start processing alle...
Start processing alxn...
Start processing amat...
Start processing amcr...
Start processing amd...
Start processing ame...
Start processing amg...
Start processing amgn...
Start processing amp...
Start processing amt...
Start processing amzn...
Start

### Define factors

Here's what we're gonna do:
1. define the functions to need to derive the factors from fundamental data.
2. using features in pandas to calculate factors all at once.
3. concatenate data of all symbols into one single sheet.

One thing needed to be mentioned, that we need to add `.shift(1)` for most of the daily data. The reason is that, the idea of factor analysis is to use today's after market data to find out the relationship with tomorrow's stock price change. So we need to shift the factor data one day forward to match the price change.

#### Quality factors

In [24]:
def EPTTM(df, qdf):
    '''
    Description: Reciprocal of pe_ratio TTM (For Trailing Twelve Months)
    Method: Earning / Market Value
    '''
    qdf.sort_values('date', inplace=True)
    
    fields = ['date']
    quarter_fields = ['date', 'statementData.incomeStatement.eps']
    
    tmp = qdf[quarter_fields].copy(deep=True)
    
    tmp['trailing_eps'] = tmp['statementData.incomeStatement.eps'].rolling(4).mean()

    tmp = pd.merge(df, tmp[['date', 'trailing_eps']], on='date', how='outer').sort_values('date')
    tmp['trailing_eps'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]
    
    tmp['ep_ratio'] = tmp['trailing_eps'] / tmp['adjClose']

    return tmp['ep_ratio']

def SPTTM(df, qdf):
    '''
    Description: Reciprocal of ps_ratio TTM (For Trailing Twelve Months)
    Method: Sales Income / Market Value
    '''
    qdf.sort_values('date', inplace=True)
    
    fields = ['date']
    quarter_fields = ['date', 'statementData.overview.rps']
    
    tmp = qdf[quarter_fields].copy(deep=True)
    
    tmp['trailing_rps'] = tmp['statementData.overview.rps'].rolling(4).mean()

    tmp = pd.merge(df, tmp[['date', 'trailing_rps']], on='date', how='outer').sort_values('date')
    tmp['trailing_rps'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]
    
    tmp['sp_ratio'] = tmp['trailing_rps'] / tmp['adjClose']

    return tmp['sp_ratio']

def roe_ttm(df, qdf):
    '''
    Description: ROE TTM (For Trailing Twelve Months)
    Method: ROE.mean() for the last 4 seasons
    '''
    qdf.sort_values('date', inplace=True)
    
    field = ['date']
    quarter_field = ['date', 'statementData.overview.roe']
    window = 20
    quarter_window = 4
    
    tmp = qdf[quarter_field].copy(deep=True)
    
    tmp['ROE_TTM'] = tmp['statementData.overview.roe'].rolling(quarter_window).sum()
        
    tmp = pd.merge(df, tmp[['date', 'ROE_TTM']], on='date', how='outer').sort_values('date')
    tmp['ROE_TTM'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]

    return tmp['ROE_TTM']

def roa_ttm(df, qdf):
    '''
    Description: ROA TTM (For Trailing Twelve Months)
    Method: ROA.mean() for the last 4 seasons
    '''
    qdf.sort_values('date', inplace=True)
    
    field = []
    quarter_field = ['date', 'statementData.overview.roa']
    window = 20
    quarter_window = 4
    
    tmp = qdf[quarter_field].copy(deep=True)
    
    tmp['ROA_TTM'] = tmp['statementData.overview.roa'].rolling(quarter_window).sum()
        
    tmp = pd.merge(df, tmp[['date', 'ROA_TTM']], on='date', how='outer').sort_values('date')
    tmp['ROA_TTM'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]

    return tmp['ROA_TTM']

def book_to_price_ratio(df, qdf):
    '''
    Description: Book to price ratio
    Method: 1 / pbRatio
    '''
    fields = ['date', 'pbRatio']
    
    tmp = df[fields].copy(deep=True)
    tmp['book_to_price'] = (1 / tmp['pbRatio']).shift(1)

    return tmp['book_to_price']

#### Growth factors

In [25]:
def SUE0(df, qdf):
    '''
    Description: Standardized unexpected profit
    Method: (actual net profit in one season - expected net profit)/ expected net profit standard deviation
    '''
    qdf.sort_values('date', inplace=True)
    
    fields = ['date']
    quarter_fields = ['date', 'statementData.incomeStatement.revenue', 'statementData.incomeStatement.costRev']
    
    tmp = qdf[quarter_fields].copy(deep=True)
    tmp['net_revenue'] = tmp['statementData.incomeStatement.revenue'] - tmp['statementData.incomeStatement.costRev']
    tmp['previous_quarter'] = tmp['net_revenue'].shift(1)
    tmp['this_quarter_shift_4'] = tmp['net_revenue'].shift(4)
    tmp['previous_quarter_shift_4'] = tmp['previous_quarter'].shift(4)
    tmp['C'] = (tmp['previous_quarter'] - tmp['previous_quarter_shift_4']).rolling(4).mean()
    tmp['epsilon'] = (tmp['previous_quarter'] - tmp['previous_quarter_shift_4']).rolling(4).std()
    tmp['Q'] = tmp['this_quarter_shift_4'] + tmp['C'] + tmp['epsilon']
    tmp['SUE0'] = (tmp['net_revenue'] - tmp['Q']) / tmp['epsilon']
    
    tmp = pd.merge(df, tmp[['date', 'SUE0']], on='date', how='outer').sort_values('date')
    tmp['SUE0'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]

    return tmp['SUE0']


def SUR0(df, qdf):
    '''
    Description: Standardized unexpected revenue
    Method: (actual operating income in one season - expected operating income)/ expected operating income standard deviation
    '''
    qdf.sort_values('date', inplace=True)
    
    fields = ['date']
    quarter_fields = ['date', 'statementData.incomeStatement.opinc']
    
    tmp = qdf[quarter_fields].copy(deep=True)
    tmp['previous_quarter'] = tmp['statementData.incomeStatement.opinc'].shift(1)
    tmp['this_quarter_shift_4'] = tmp['statementData.incomeStatement.opinc'].shift(4)
    tmp['previous_quarter_shift_4'] = tmp['previous_quarter'].shift(4)
    tmp['C'] = (tmp['previous_quarter'] - tmp['previous_quarter_shift_4']).rolling(4).mean()
    tmp['epsilon'] = (tmp['previous_quarter'] - tmp['previous_quarter_shift_4']).rolling(4).std()
    tmp['Q'] = tmp['this_quarter_shift_4'] + tmp['C'] + tmp['epsilon']
    tmp['SUR0'] = (tmp['statementData.incomeStatement.opinc'] - tmp['Q']) / tmp['epsilon']
    
    tmp = pd.merge(df, tmp[['date', 'SUR0']], on='date', how='outer').sort_values('date')
    tmp['SUR0'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]

    return tmp['SUR0']

def DELTAROE(df, qdf):
    '''
    Description: Quarter-on-quarter ROE change ratio
    Method: ROE this quarter - ROA same quarter last year
    '''
    qdf.sort_values('date', inplace=True)
    
    fields = ['date']
    quarter_fields = ['date', 'statementData.overview.roe']
    
    tmp = qdf[quarter_fields].copy(deep=True)
    tmp['last_year_roe'] = tmp['statementData.overview.roe'].shift(4)
    tmp['DELTAROE'] = tmp['statementData.overview.roe'] - tmp['last_year_roe']

    tmp = pd.merge(df, tmp[['date', 'DELTAROE']], on='date', how='outer').sort_values('date')
    tmp['DELTAROE'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]

    return tmp['DELTAROE']


def DELTAROA(df, qdf):
    '''
    Description: Quarter-on-quarter ROA ratio
    Method: ROA this quarter - ROA same quarter last year
    '''
    qdf.sort_values('date', inplace=True)
    
    fields = ['date']
    quarter_fields = ['date', 'statementData.overview.roa']
    
    tmp = qdf[quarter_fields].copy(deep=True)
    tmp['last_year_roa'] = tmp['statementData.overview.roa'].shift(4)
    tmp['DELTAROA'] = tmp['statementData.overview.roa'] - tmp['last_year_roa']

    tmp = pd.merge(df, tmp[['date', 'DELTAROA']], on='date', how='outer').sort_values('date')
    tmp['DELTAROA'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]

    return tmp['DELTAROA']

def net_profit_growth_rate(df, qdf):
    '''
    Description: annual net profit growth rate
    Method: (Net profit this year TTM / net profit last year TTM) - 100%
    '''
    qdf.sort_values('date', inplace=True)
    
    fields = ['date']
    quarter_fields = ['date', 'year', 'quarter', 'statementData.incomeStatement.revenue', 'statementData.incomeStatement.costRev']
    quarter_window = 4
    
    tmp = qdf[quarter_fields].copy(deep=True)
    tmp['net_profit'] = tmp['statementData.incomeStatement.revenue'] - tmp['statementData.incomeStatement.costRev']
    tmp['net_profit_TTM'] = tmp['net_profit'].rolling(quarter_window).sum()
    tmp['net_profit_TTM_shift_4'] = tmp['net_profit_TTM'].shift(quarter_window)
    tmp['net_profit_growth_rate'] = (tmp['net_profit_TTM'] / tmp['net_profit_TTM_shift_4']) - 1

    tmp = pd.merge(df, tmp[['date', 'net_profit_growth_rate']], on='date', how='outer').sort_values('date')
    tmp['net_profit_growth_rate'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]

    return tmp['net_profit_growth_rate']

def operating_revenue_growth_rate(df, qdf):
    '''
    Description: annual operating revenue growth rate
    Method: (operating revenue this year TTM / operating revenue last year TTM) - 100%
    '''
    qdf.sort_values('date', inplace=True)
    
    fields = ['date']
    quarter_fields = ['date', 'year', 'quarter', 'statementData.incomeStatement.opinc']
    quarter_window = 4
    
    tmp = qdf[quarter_fields].copy(deep=True)
    tmp['revenue_TTM'] = tmp['statementData.incomeStatement.opinc'].rolling(quarter_window).sum()
    tmp['revenue_TTM_shift_4'] = tmp['revenue_TTM'].shift(quarter_window)
    tmp['total_revenue_growth_rate'] = (tmp['revenue_TTM'] / tmp['revenue_TTM_shift_4']) - 1

    tmp = pd.merge(df, tmp[['date', 'total_revenue_growth_rate']], on='date', how='outer').sort_values('date')
    tmp['total_revenue_growth_rate'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]

    return tmp['total_revenue_growth_rate']

def total_profit_growth_rate(df, qdf):
    '''
    Description: annual total revenue growth rate
    Method: (total revenue this year TTM / total revenue last year TTM) - 100%
    '''
    qdf.sort_values('date', inplace=True)
    
    fields = ['date']
    quarter_fields = ['date', 'year', 'quarter', 'statementData.incomeStatement.revenue']
    quarter_window = 4
    
    tmp = qdf[quarter_fields].copy(deep=True)
    tmp['profit_TTM'] = tmp['statementData.incomeStatement.revenue'].rolling(quarter_window).sum()
    tmp['profit_TTM_shift_4'] = tmp['profit_TTM'].shift(quarter_window)
    tmp['total_profit_growth_rate'] = (tmp['profit_TTM'] / tmp['profit_TTM_shift_4']) - 1

    tmp = pd.merge(df, tmp[['date', 'total_profit_growth_rate']], on='date', how='outer').sort_values('date')
    tmp['total_profit_growth_rate'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]

    return tmp['total_profit_growth_rate']

#### Momentum factors

In [26]:
def ILLIQ(df, qdf):
    '''
    Description: illiquidity factor
    Method: mean of the (absolute value of the price change in the past 20 days / trade volume)
    '''
    # ! We should use trading money instead of trading volume.
    # ! Trading money = the money amount that has been traded on the market
    fields = ['adjClose', 'adjVolume']
    window = 20
    tmp = df[fields].copy(deep=True)
    tmp.ffill(inplace=True)
    
    tmp['pct_change'] = df['adjClose'].pct_change()
    tmp['Vol'] = df['adjVolume']
    tmp['pct_vol'] = tmp['pct_change'] / tmp['Vol']
    tmp['ILLIQ'] = tmp['pct_vol'].rolling(window).mean().shift(1)

    return tmp['ILLIQ']

def VOL20(df, qdf):
    '''
    Description: 20-day turnover average
    Method: average of turnover in past 20 days
    '''
    qdf.sort_values('date', inplace=True)
    
    field = ['adjVolume']
    quarter_field = ['date', 'statementData.balanceSheet.sharesBasic']
    window = 20
    tmp = qdf[quarter_field].copy(deep=True)
        
    tmp = pd.merge(df, tmp[['date', 'statementData.balanceSheet.sharesBasic']], on='date', how='outer').sort_values('date')
    tmp['statementData.balanceSheet.sharesBasic'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]
    
    tmp['vol'] = tmp['adjVolume'] / tmp['statementData.balanceSheet.sharesBasic']
    tmp['vol20'] = tmp['vol'].rolling(window).sum().shift(1)

    return tmp['vol20']


def VOL60(df, qdf):
    '''
    Description: 60-day turnover average
    Method: average of turnover in past 60 days
    '''
    field = ['adjVolume']
    quarter_field = ['date', 'statementData.balanceSheet.sharesBasic']
    window = 60
    tmp = qdf[quarter_field].copy(deep=True)
        
    tmp = pd.merge(df, tmp[['date', 'statementData.balanceSheet.sharesBasic']], on='date', how='outer').sort_values('date')
    tmp['statementData.balanceSheet.sharesBasic'].ffill(inplace=True)
    tmp = tmp[~pd.isna(tmp['close'])]
    
    tmp['vol'] = tmp['adjVolume'] / tmp['statementData.balanceSheet.sharesBasic']
    tmp['vol60'] = tmp['vol'].rolling(window).sum().shift(1)

    return tmp['vol60']

#### Volatility factors

In [27]:
def ATR1M(df, qdf):
    '''
    Description: Average true range of the past month
    Method: Calculate the ATR indicator in the past 20 days
    '''

    fields = ['date','adjHigh', 'adjLow', 'adjClose']
    window = 20
    tmp = df[fields].copy(deep=True)
    tmp.ffill(inplace=True)
    
    tmp['high'] = tmp['adjHigh'].shift(1)
    tmp['low'] = tmp['adjLow'].shift(1)
    tmp['close'] = tmp['adjClose'].shift(1)
    
    tmp['hml'] = tmp['high'] - tmp['low']
    tmp['cmh'] = (tmp['close'].shift(1) - tmp['high']).abs()
    tmp['cml'] = (tmp['close'].shift(1) - tmp['low']).abs()
    
    tmp['TR'] = tmp[['hml', 'cmh', 'cml']].max(axis=1, skipna=False)
    tmp['ATR1M'] = tmp['TR'].rolling(window).mean().shift(1)
    
    return tmp['ATR1M']
    
def ATR3M(df, qdf):
    '''
    
    Description: Average true range of the past three months
    Method: Calculate the ATR indicator in the past 60 days
    '''

    fields = ['date','adjHigh', 'adjLow', 'adjClose']
    window = 60
    tmp = df[fields].copy(deep=True)
    tmp.fillna(0, inplace=True)
    
    tmp['high'] = tmp['adjHigh'].shift(1)
    tmp['low'] = tmp['adjLow'].shift(1)
    tmp['close'] = tmp['adjClose'].shift(1)
    
    tmp['hml'] = tmp['high'] - tmp['low']
    tmp['cmh'] = (tmp['close'].shift(1) - tmp['high']).abs()
    tmp['cml'] = (tmp['close'].shift(1) - tmp['low']).abs()
    
    tmp['TR'] = tmp[['hml', 'cmh', 'cml']].max(axis=1, skipna=False)
    tmp['ATR3M'] = tmp['TR'].rolling(window).mean().shift(1)
    
    return tmp['ATR3M']

#### Size factors

In [28]:
def natural_log_of_market_cap(df, qdf):
    '''
    Description: natural of the market capital
    Method: log(marketCap)
    '''
    fields = ['date', 'marketCap']
    
    tmp = df[fields].copy(deep=True)
    tmp['ln_of_market_cap'] = tmp['marketCap'].apply(np.log).shift(1)
                              
    return tmp['ln_of_market_cap']

#### Technical factors

In [29]:
def ROC20(df, qdf):
    '''
    Description: Price rate of change in 20 days
    Method: 
        1. AX=today's close - 20 days ago close price
        2. BX=20 days ago close price
        3. ROC=AX/BX*100%
    '''

    fields = ['date', 'adjClose']

    window = 20
    tmp = df[fields].copy(deep=True)
    tmp.fillna(0, inplace=True)
    
    tmp['BX'] = tmp['adjClose'].shift(window)
    tmp['AX'] = tmp['adjClose'] - tmp['BX']
    tmp['ROC20'] = ((tmp['AX'] / tmp['BX'])*100).shift(1)

    return tmp['ROC20']


def ROC60(df, qdf):
    '''
    Description: Price rate of change in 60 days
    Method: 
        1. AX=today's close - 60 days ago close price
        2. BX=60 days ago close price
        3. ROC=AX/BX*100%
    '''

    fields = ['date', 'adjClose']

    window = 60
    tmp = df[fields].copy(deep=True)
    tmp.fillna(0, inplace=True)
    
    tmp['BX'] = tmp['adjClose'].shift(window)
    tmp['AX'] = tmp['adjClose'] - tmp['BX']
    tmp['ROC60'] = ((tmp['AX'] / tmp['BX'])*100).shift(1)

    return tmp['ROC60']

#### Fundamental factors

In [30]:
def get_market_cap(df, qdf):
    '''
    Description: marketCap directly from financial report
    Method: marketCap
    '''
    fields = ['date', 'marketCap']
                              
    return df['marketCap']

def get_adj_close(df, qdf):
    '''
    Description: close price directly from financial report
    Method: use adjClose price
    '''
    fields = ['date', 'adjClose']
                              
    return df['adjClose']

### Get factor data

#### Construct factors

In [31]:
# Aggregate the factor functions that we have defined
alpha_factors = [
    ATR1M, ATR3M, ILLIQ, DELTAROA, DELTAROE, SUR0, SUE0, SPTTM, EPTTM,
    natural_log_of_market_cap, book_to_price_ratio, ROC20, ROC60, net_profit_growth_rate, 
    operating_revenue_growth_rate, total_profit_growth_rate, roe_ttm, roa_ttm, VOL20, VOL60
]

risk_factors = [
    get_market_cap, get_adj_close
]

factors = list(set(alpha_factors + risk_factors))

# Initiate a DataFrame to contain the final data
result_df = None

# Start enumerate through all symbols in the stock list
for symbol in notebook.tqdm(stock_list):
    df, dd = load_data(
        symbol=symbol,
        start_date=START_DATE,
        end_date=END_DATE,
        freq='daily',
        path=DATA_PATH
    )
            
    tmp = df['date'].copy(deep=True)
    try:
        # Loop through the factor functions in the enumeration 
        for func in factors:
            tmp = pd.concat([tmp, func(df, dd)], axis=1)
    except Exception as e:
        print(symbol)
        print(e)
        continue

    # Dealing with dtype
    tmp['sicCode'] = int(meta_df[meta_df['ticker'] == symbol]['sicCode'].iloc[0])
    tmp['sector'] = meta_df[meta_df['ticker'] == symbol]['sector'].values[0]
    tmp['ticker'] = symbol
    tmp.dropna(axis=0, inplace=True)
    
    if result_df is None:
        result_df = tmp
    else:
        # Append data of one symbol to the final DataFrame
        result_df = result_df.append(tmp)

# Redefine data type
result_df['date'] = pd.to_datetime(result_df['date'], utc=True)
result_df = result_df.sort_values(['date', 'ticker']).set_index(['date', 'ticker'])

# result_df.to_csv('/Users/michael/Desktop/result.csv')
print(f'Data has been saved as \'~/Desktop/result.csv\' with shape {result_df.shape}')

vnt
"['statementData.overview.roe'] not in index"

Data has been saved as '~/Desktop/result.csv' with shape (109903, 24)


In [32]:
result_df

DELTAROE  ln_of_market_cap      ROC60  \
date                      ticker                                          
2020-01-24 00:00:00+00:00 mdt     0.023287         25.788632  15.279062   
                          ntap   -1.266889         23.308636   8.968357   
2020-01-27 00:00:00+00:00 amat   -0.052044         24.748587   9.250699   
                          cpb    -0.529817         23.382802   5.550597   
                          csco    0.018104         26.022862   4.310172   
...                                    ...               ...        ...   
2021-02-12 00:00:00+00:00 kdp    -0.012643         24.527825   9.421566   
                          nktr    0.076847         22.129686  31.731326   
                          wh      0.254887         22.459466  11.982180   
2021-02-16 00:00:00+00:00 kdp    -0.012643         24.523428   8.386616   
                          wh      0.254887         22.447605   8.429592   

                                  DELTAROA  sp_ratio  net_profit_growth_rate  \
date                      ticker                                               
2020-01-24 00:00:00+00:00 mdt     0.013871  0.049689               -0.003072   
                          ntap    0.035883  0.102383               -0.060790   
2020-01-27 00:00:00+00:00 amat   -0.026779  0.069264               -0.091549   
                          cpb    -0.130555  0.141557                0.190704   
                          csco    0.000197  0.065455                0.043513   
...                                    ...       ...                     ...   
2021-02-12 00:00:00+00:00 kdp    -0.006275  0.063930                0.066013   
                          nktr    0.052242  0.010057                0.454668   
                          wh      0.060344  0.057898               -0.397436   
2021-02-16 00:00:00+00:00 kdp    -0.006275  0.064642                0.066013   
                          wh      0.060344  0.057421               -0.397436   

                                     ATR1M   ROE_TTM  \
date                      ticker                       
2020-01-24 00:00:00+00:00 mdt     1.468760  0.378975   
                          ntap    1.198524  4.186325   
2020-01-27 00:00:00+00:00 amat    1.254684  1.450716   
                          cpb     0.566922  1.514039   
                          csco    0.657420  1.238254   
...                                    ...       ...   
2021-02-12 00:00:00+00:00 kdp     0.698750  0.213495   
                          nktr    1.317925 -1.271269   
                          wh      1.726750 -0.099956   
2021-02-16 00:00:00+00:00 kdp     0.698995  0.213495   
                          wh      1.840250 -0.099956   

                                  total_revenue_growth_rate         ILLIQ  \
date                      ticker                                            
2020-01-24 00:00:00+00:00 mdt                      0.027360  6.952166e-10   
                          ntap                    -0.107481 -1.729182e-10   
2020-01-27 00:00:00+00:00 amat                    -0.167304  6.694108e-11   
                          cpb                      0.082699  1.020245e-10   
                          csco                     0.049348  6.917694e-11   
...                                                     ...           ...   
2021-02-12 00:00:00+00:00 kdp                      0.240299 -1.395300e-10   
                          nktr                    -0.015473  5.534351e-09   
                          wh                      -1.150820  1.959715e-09   
2021-02-16 00:00:00+00:00 kdp                      0.240299 -1.579897e-10   
                          wh                      -1.150820  2.449902e-10   

                                     ATR3M  ep_ratio      SUE0   ROA_TTM  \
date                      ticker                                           
2020-01-24 00:00:00+00:00 mdt     1.574542  0.008495 -0.592085  0.211441   
                          ntap    1.263933  0.018639 -0.97

Cool! We have all the factors data we need!